# Assignment 3

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- sklearn [link](https://scikit-learn.org/)
- Keras [link](https://keras.io/guides/)

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q3.ipynb`.


## Question 3
Fake news is a widespread problem and there are many methods for combating it.
You have to build a fake news detection system using a ML model. Train any ML model (ANN, LSTM) over the given Dataset.
The dataset has short statements spoken by people and has the meta-information and corresponding label for those sentences. 
Your target is label column which has 6 labels(in the increasing order of truthfullness): pants-fire, false, barely-true, half-true, mostly-true, true.

The features are 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c', 'venue' and the target is column "label".

The statement is made by speaker whose job, party are given along with 6 columns which are an account of the  type of news(labels) the person has shared before. 
The person who has shared fake content before is likely to share it in future and this can be accounted by the ML model as a feature. Column barely_true_c contains how many barely_true news has the speaker shared (and so is with column X_c, value of X_c is number of X the person shared).


You have to perform two tasks:
* task1: Binary classification <br>
Classify the given news as true/false. Take the labels pants-fire, false, barely-true as false and rest (half-true, mostly-true, true) as true.
* task2: Six-way classification <br>
Classify the given news into six-classes "pants-fire, false, barely-true, half-true, mostly-true, true".

For each of the tasks:
1) Experiment with depth of network and try to fine-tune hyperparameters reporting your observations. <br>
2) Report the accuracy, f1-score, confusion matrix on train, val and test sets. <br>
3) Experiment with bag-of-words, glove and bert embeddings(code given in the below notebook) and report results. <br> Comment on what is the affect of embedding on the results.

The pre-processing code is provided, you need to write the training and test.

Note: You are supposed to train on trainset, fine-tune on val and just eval on test set. If found that you trained on val/test sets, the penalty will be incurred.

In [ ]:
# !pip install numpy
# !pip install tensorflow
# !pip install re
# !pip install nltk
# !pip install keras
# !pip install sklearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
from tensorflow import keras  #feel free to use any other library
import numpy as np

import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import np_utils


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train = pd.read_csv('drive/MyDrive/q3_data/train.csv')
val = pd.read_csv('drive/MyDrive/q3_data/val.csv')
test = pd.read_csv('drive/MyDrive/q3_data/test.csv')

In [ ]:
# Dropping the 'id' column
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
val.drop('id', axis = 1, inplace = True)

In [ ]:
train.head(5)

,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [ ]:
# Checking the shape of data
print(train.shape)
print(val.shape)
print(test.shape)

(10269, 13)
(1284, 13)
(1283, 13)


## Clean and pre-process data
* Replace missing values
* Remove numbers and special characters
* Convert to upper-case

We experiment with two types of processing, one directly appending the other attributes like subject, job, state, party to sentence and then applying bag of words on it.

Other being encoding sentence with glove embeddings and passing just that.

In [ ]:

def dataPreprocessing(data):
    '''Function for cleaning the dataset
    '''
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = statement.split()
        
        # you can experiment with any other stemmers
        ps = PorterStemmer()
        statement = [ps.stem(word) for word in statement if not word in set(stopwords.words('english'))] # Stemming the dataset and removing stopwords
        statement = ' '.join(statement)
        subject = data['subject'][x].replace(',', ' ')
        speaker = data['speaker'][x]
        job = data['job'][x].lower()
        # job = job.replace(' ', '-')
        state = data['state'][x].lower()
        party = data['party'][x].lower()
        corpus.append(statement + ' '  + subject + ' ' + job + ' ' + state + ' ' + party)
    return corpus

In [ ]:
x_train = dataPreprocessing(train)
x_val = dataPreprocessing(val) 
x_test = dataPreprocessing(test) 

In [ ]:
len(x_train), len(x_val), len(x_test)

(10269, 1284, 1283)

In [ ]:
corpus = x_train + x_val + x_test

## Using bag-of-words embedding


In [ ]:
# Converting the corpus into bag-of-words
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus).toarray()

In [ ]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
X.shape

(12836, 8000)

In [ ]:
train.columns

Index(['label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
       'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

In [ ]:
# Selecting the columns 'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c'
label_cols = ['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c']
x_train2 = train[label_cols]
x_val2 = val[label_cols]
x_test2 = test[label_cols]

In [ ]:
x_train2

,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c
0,0,1,0,0,0
1,0,0,1,1,0
2,70,71,160,163,9
3,7,19,3,5,44
4,15,9,20,19,2
...,...,...,...,...,...
10264,0,1,1,1,0
10265,0,0,0,1,0
10266,4,11,5,3,3
10267,3,1,3,0,0


In [ ]:
# Stacking x_train and x_train2 horizontally
x_train_bow = np.hstack((X[:len(x_train)], x_train2))
x_val_bow = np.hstack((X[len(x_train):len(x_train)+len(x_val)], x_val2))
x_test_bow = np.hstack((X[len(x_train)+len(x_val):], x_test2))

In [ ]:
x_train_bow.shape

(10269, 8005)

## Use of Glove Embedding


download glove embeddings from 'https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip'
and place in your current working folder


In [ ]:
# !wget "https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip"
!unzip "glove.6B.zip" -d "glove"

unzip:  cannot find or open glove.6B.zip, glove.6B.zip.zip or glove.6B.zip.ZIP.


In [ ]:
emmbed_dict = {}
with open('drive/MyDrive/glove/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector


In [ ]:
emmbed_dict['oov'] = np.zeros(200)

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def dataPreprocessing_glove(data):
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = word_tokenize(statement)
        
        if(len(statement)<40):
          tmp = len(statement)
          while tmp < 40:
            statement.append('ashishrai')
            tmp += 1
        else:
          statement = statement[:40]

        embed_statement = []
        for w in statement:
            if w in emmbed_dict:
                embed_statement.append(emmbed_dict[w])
            else:
                embed_statement.append(emmbed_dict['oov'])
         
        # bonus: Think how you can encode the below features(hint: look upon label encoding or training your own word2vec or any other embedding model)
    
#         subject = data['subject'][x].replace(',', ' ')
#         speaker = data['speaker'][x]
#         job = data['job'][x].lower()
#         # job = job.replace(' ', '-')
#         state = data['state'][x].lower()
#         party = data['party'][x].lower()
        corpus.append(embed_statement)
    
    return np.array(corpus)    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
x_train_glove = dataPreprocessing_glove(train)
x_val_glove = dataPreprocessing_glove(val) 
x_test_glove = dataPreprocessing_glove(test) 

In [ ]:
# x_train_glove = np.asarray(x_train_glove)
# print(x_train_glove.shape)

In [ ]:
x_train_glove = np.hstack((x_train_glove.reshape(-1,8000), x_train2))
x_val_glove = np.hstack((x_val_glove.reshape(-1,8000), x_val2))
x_test_glove = np.hstack((x_test_glove.reshape(-1,8000), x_test2))

## Use of bert embeddings
note: we used our pre-processed code for bow which has the attributed appended to end the end of sentence. 

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

x_train_bert = np.hstack((model.encode(x_train), x_train2))
x_val_bert = np.hstack((model.encode(x_val), x_val2))
x_test_bert = np.hstack((model.encode(x_test), x_test2))

Now use the above 3 types of embedded inputs(bow, glove, bert embeddings) for the 2 classification tasks and compare their outputs


# Six-way classification

## Preprocessing

In [ ]:
num_classes = 6
# Preprocessing function for the labels
def categorize(data):
    y = data["label"].tolist()

    # Encoding the Dependent Variable
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)

    # Converting to binary class matrix
    y = np_utils.to_categorical(y, num_classes)
    return y

In [ ]:
y_train_six_way = categorize(train)
y_test_six_way = categorize(test)
y_val_six_way = categorize(val)

Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


## Model

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense

# class LSTM_Model:
#   def __init__(self, input, output, units ):
#     self.model = Sequential()
#     self.model.add(Input(input))
#     self.model.add(LSTM(units=units))
#     self.model.add(Dense(units=output, activation='softmax'))
#     self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#   def print_summary(self):
#     print(self.model.summary())

#   def train(self, x_train, y_train, validation_data, batch_size=64, epochs=3):
#     self.model.fit(x_train, y_train, validation_data=validation_data, batch_size=batch_size, epochs=epochs)
#     return self.model

#   def model_evaluate(model, x_test, y_test):
#     y_test = np.argmax(y_test, axis=1)
#     y_pred = model.predict(x_test)
#     y_pred = np.argmax(y_pred)
#     acc = accuracy_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred, average='micro')
#     con_mat = confusion_matrix(y_test, y_pred)
#     print("accuracy: ", acc)
#     print("f1 score", f1_score)
#     print("Confusion Matrix", con_mat)


def print_metrics(y_test, y_pred):
  acc = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred, average="micro")
  con_mat = confusion_matrix(y_test, y_pred)
  print("accuracy: ", acc)
  print("f1 score", f1)
  print("Confusion Matrix: \n", con_mat)


### Six way classification: **Bag of words**

In [ ]:
!pip install keras
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

model = Sequential()
model.add(Input((8005,1)))
model.add(LSTM(units=100))
model.add(Dense(units=6, activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_bow = model.fit(x_train_bow, y_train_six_way, validation_data=(x_val_bow, y_val_six_way), batch_size=64, epochs=10)

y_test = np.argmax(y_test_six_way, axis=1)
y_pred = model.predict(x_test_bow, verbose=0)
y_pred = np.argmax(y_pred, axis=1)
print_metrics(y_test, y_pred)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 100)               40800     
                                                                 
 dense_7 (Dense)             (None, 6)                 606       
                                                                 
Total params: 41,406
Trainable params: 41,406
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
161/161 [==============================] - 64s 373ms/step - loss: 1.7319 - accuracy: 0.2306 - val_loss: 1.7091 - val_accuracy: 0.2305
Epoch 2/10
161/161 [==============================] - 56s 350ms/step - loss: 1.7516 - accuracy: 0.2679 - val_loss: 1.8802 - val_accuracy: 0.1846
Epoch 3/10
161/161 [==============================] - 52s 320ms/step - loss: 1.7448 - accuracy: 0.2235 - val_loss: 1.7191 - val_accuracy: 0.2453
Epoch 4/10


In [ ]:
# model_bow = model.fit(x_train_bow, y_train_six_way, validation_data=(x_val_bow, y_val_six_way), batch_size=64, epochs=3)

In [ ]:
# y_test = np.argmax(y_test_six_way, axis=1)
# y_pred = model.predict(x_test_bow, verbose=0)
# y_pred = np.argmax(y_pred, axis=1)
# print_metrics(y_test, y_pred)

### Six way classification: **Glove**

In [ ]:
model = Sequential()
model.add(Input((8005,1)))
model.add(LSTM(units=100))
model.add(Dense(units=6, activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_bow = model.fit(x_train_glove, y_train_six_way, validation_data=(x_val_glove, y_val_six_way), batch_size=64, epochs=10)

y_test = np.argmax(y_test_six_way, axis=1)
y_pred = model.predict(x_test_glove, verbose=0)
y_pred = np.argmax(y_pred, axis=1)
print_metrics(y_test, y_pred)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 100)               40800     
                                                                 
 dense_8 (Dense)             (None, 6)                 606       
                                                                 
Total params: 41,406
Trainable params: 41,406
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
161/161 [==============================] - 67s 403ms/step - loss: 1.7308 - accuracy: 0.2294 - val_loss: 1.7057 - val_accuracy: 0.2812
Epoch 2/10
161/161 [==============================] - 53s 328ms/step - loss: 1.6842 - accuracy: 0.3010 - val_loss: 1.6655 - val_accuracy: 0.3902
Epoch 3/10
161/161 [==============================] - 51s 318ms/step - loss: 1.6633 - accuracy: 0.3594 - val_loss: 1.6455 - val_accuracy: 0.3785
Epoch 4/10


### Six way classification: **Bert**

In [ ]:
model = Sequential()
model.add(Input((389,1)))
model.add(LSTM(units=100))
model.add(Dense(units=6, activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_bow = model.fit(x_train_bert, y_train_six_way, validation_data=(x_val_bert, y_val_six_way), batch_size=64, epochs=10)

y_test = np.argmax(y_test_six_way, axis=1)
y_pred = model.predict(x_test_bert, verbose=0)
y_pred = np.argmax(y_pred, axis=1)
print_metrics(y_test, y_pred)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 100)               40800     
                                                                 
 dense_9 (Dense)             (None, 6)                 606       
                                                                 
Total params: 41,406
Trainable params: 41,406
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
161/161 [==============================] - 5s 24ms/step - loss: 1.7257 - accuracy: 0.2296 - val_loss: 1.6875 - val_accuracy: 0.2757
Epoch 2/10
161/161 [==============================] - 5s 31ms/step - loss: 1.6159 - accuracy: 0.3234 - val_loss: 1.4983 - val_accuracy: 0.4136
Epoch 3/10
161/161 [==============================] - 6s 36ms/step - loss: 1.4195 - accuracy: 0.4253 - val_loss: 1.3705 - val_accuracy: 0.4393
Epoch 4/10
161/16

# Binary Classification

## Preprocessing

In [ ]:
num_classes = 2

In [ ]:
# Function for preprocessing labels
def dataPreprocessingBinary(data):
    y = data["label"].tolist()

    # Changing the 'half-true', 'mostly-true', barely-true', 'pants-fire' labels to True/False for Binary Classification
    for x in range(len(y)):
        if(y[x] == 'half-true'):
            y[x] = 'True'
        elif(y[x] == 'mostly-true'):
            y[x] = 'True'
        elif(y[x] == 'barely-true'):
            y[x] = 'False'
        elif(y[x] == 'pants-fire'):
            y[x] = 'False'

    # Converting the lables into binary class matrix
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)
    y = np_utils.to_categorical(y, num_classes)
    return y

In [ ]:
y_train_binary = dataPreprocessingBinary(train)
y_test_binary = dataPreprocessingBinary(test)
y_val_binary = dataPreprocessingBinary(val)

## Model
Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


In [ ]:
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

### Binary classification: **Bag of words**

In [ ]:
model = Sequential()
model.add(Input((8005,1)))
model.add(LSTM(units=100))
model.add(Dense(units=2, activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train_bow, y_train_binary, validation_data=(x_val_bow, y_val_binary), batch_size=64, epochs=10)

y_test = np.argmax(y_test_binary, axis=1)
y_pred = model.predict(x_test_bow, verbose=0)
y_pred = np.argmax(y_pred, axis=1)
print_metrics(y_test, y_pred)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 100)               40800     
                                                                 
 dense_10 (Dense)            (None, 2)                 202       
                                                                 
Total params: 41,002
Trainable params: 41,002
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
161/161 [==============================] - 56s 338ms/step - loss: 0.6578 - accuracy: 0.6040 - val_loss: 0.6373 - val_accuracy: 0.6012
Epoch 2/10
161/161 [==============================] - 53s 331ms/step - loss: 0.6058 - accuracy: 0.6852 - val_loss: 0.5964 - val_accuracy: 0.6986
Epoch 3/10
161/161 [==============================] - 51s 319ms/step - loss: 0.5798 - accuracy: 0.7100 - val_loss: 0.5845 - val_accuracy: 0.6924
Epoch 4/10

### Binary classification: **Glove**

In [ ]:
model = Sequential()
model.add(Input((8005,1)))
model.add(LSTM(units=100))
model.add(Dense(units=2, activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train_glove, y_train_binary, validation_data=(x_val_glove, y_val_binary), batch_size=64, epochs=10)

y_test = np.argmax(y_test_binary, axis=1)
y_pred = model.predict(x_test_glove, verbose=0)
y_pred = np.argmax(y_pred, axis=1)
print_metrics(y_test, y_pred)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 100)               40800     
                                                                 
 dense_11 (Dense)            (None, 2)                 202       
                                                                 
Total params: 41,002
Trainable params: 41,002
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
161/161 [==============================] - 53s 320ms/step - loss: 0.6536 - accuracy: 0.6153 - val_loss: 0.6252 - val_accuracy: 0.6425
Epoch 2/10
161/161 [==============================] - 51s 316ms/step - loss: 0.5985 - accuracy: 0.6909 - val_loss: 0.5921 - val_accuracy: 0.7056
Epoch 3/10
161/161 [==============================] - 51s 315ms/step - loss: 0.5747 - accuracy: 0.7126 - val_loss: 0.5701 - val_accuracy: 0.6955
Epoch 4/10

### Binary classification: **Bert**

In [ ]:
model = Sequential()
model.add(Input((389,1)))
model.add(LSTM(units=100))
model.add(Dense(units=2, activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train_bert, y_train_binary, validation_data=(x_val_bert, y_val_binary), batch_size=64, epochs=10)

y_test = np.argmax(y_test_binary, axis=1)
y_pred = model.predict(x_test_bert, verbose=0)
y_pred = np.argmax(y_pred, axis=1)
print_metrics(y_test, y_pred)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 100)               40800     
                                                                 
 dense_12 (Dense)            (None, 2)                 202       
                                                                 
Total params: 41,002
Trainable params: 41,002
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
161/161 [==============================] - 5s 23ms/step - loss: 0.6587 - accuracy: 0.6085 - val_loss: 0.6348 - val_accuracy: 0.6441
Epoch 2/10
161/161 [==============================] - 3s 19ms/step - loss: 0.6111 - accuracy: 0.6636 - val_loss: 0.5908 - val_accuracy: 0.6760
Epoch 3/10
161/161 [==============================] - 3s 20ms/step - loss: 0.5602 - accuracy: 0.7085 - val_loss: 0.5621 - val_accuracy: 0.6893
Epoch 4/10
161/1